In [2]:
from binance.spot import Spot as sp
import inspect as i
import json
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pytz
from plotly.subplots import make_subplots
from finta import TA as ta

tz_local = pytz.timezone("Europe/Zurich")

apiKey = "OY6oBQD3qm6VGzokI29UaCm8hrnpCrjv4KEpCY45UasLAj7D3xkLPGwFjNNz1neL"
apiKey_secret = "D82a2gKS5lGbmFRvkPP9PEQTwnfXKfxFBNddOVrjixQ8MJkSirLxuRyNoenO3JVj"

baseURL = "https://testnet.binance.vision"

pd.options.display.float_format = "{:,.3f}".format

In [3]:
#API SETUP
sp_client = sp(key=apiKey, secret=apiKey_secret)

#get Different cryptos
dict_symbols = sp_client.exchange_info()

#API CALL
api_interval = "1m"
pd_interval = "T"
api_limit = 1000
end_time = dt.datetime.now()
end_time_api = int(round(end_time.timestamp()*1000,0))

dict_data = sp_client.klines(endTime=end_time_api, symbol="BTCBUSD", interval="1m", limit=api_limit)

In [4]:
df_symbols = pd.json_normalize(dict_symbols["symbols"])

In [5]:
headers_df = ["openTime","open", "high","low","close","vol","closeTime","quiteAssetVolume","trades_qty","baseAssetVol_taker","baseQuoteVol_taker","ignore"]

df_data_raw = pd.DataFrame(dict_data, columns=headers_df)
#convert to datetimes
df_data_raw["openTime"] = pd.to_datetime(df_data_raw["openTime"]/1000, unit="s")
df_data_raw["closeTime"] = pd.to_datetime(df_data_raw["closeTime"]/1000, unit="s")

#round to nearest second
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.floor("S")

#change timezone

#reset index to opentime
df_data_raw.reset_index(inplace=True,drop=True)
df_data_raw.set_index("openTime", inplace=True)

# convert all none-date columns to float
cols = df_data_raw.select_dtypes(exclude=["datetime64"]).columns
df_data_raw[cols] = df_data_raw[cols].apply(pd.to_numeric, downcast="float",errors="coerce")

# mark TZ as UTC then change to CET
df_data_raw.index = df_data_raw.index.tz_localize("UTC").tz_convert("CET")
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.tz_localize("UTC").dt.tz_convert("CET")


In [6]:
df_data_raw.index.min().to_pydatetime().strftime("%Y-%m-%d %H:%M:%S")

'2021-08-12 15:42:00'

In [7]:
#CREATE BASE TIMELINE TO JOIN AGAINST
end_time_pd = df_data_raw.index.max().to_pydatetime().strftime("%Y-%m-%d %H:%M:%S")
df_base = pd.DataFrame(pd.date_range(end=end_time_pd, freq=pd_interval, periods=api_limit), columns=["dateTime"])

df_base.reset_index(drop=True, inplace=True)
df_base.set_index("dateTime", inplace=True)
df_base.index = df_base.index.tz_localize("CET")

In [8]:
#JOIN AGAINST BASE TIMELIME
df_data = pd.concat([df_base, df_data_raw], axis=1)
df_data.index.names = ["dateTime"]

#CREATE DATE SKKIP LIST
l_skipDates = df_data.isna().index.strftime("%Y-%m-%d %H:%M:%S").tolist()

In [9]:
#date_offset=(dt.datetime.now()-dt.timedelta(hours=24))
#date_offset
#dt64 = np.datetime64(date_offset)

dateOffset_now = tz_local.localize(dt.datetime.now(), is_dst=None)
offset_hrs = 48

df_data_dtFilter = df_data[df_data.index >= dateOffset_now - dt.timedelta(hours=offset_hrs)].copy()


In [18]:
fig = go.Figure(data=go.Candlestick(
    x=df_data_dtFilter.index
    ,open=df_data_dtFilter['open']
    ,close=df_data_dtFilter["close"]
    ,high=df_data_dtFilter["high"]
    ,low=df_data_dtFilter["low"]
))                             


fig.update_xaxes(rangebreaks=[dict(values=l_skipDates, dvalue=(60* 1000))]
)

fig.show()

In [11]:
df_hourly = df_data[df_data.index > "2021-08-06"].copy()
dailyClose = df_data_dtFilter[['close']]
dailyPctChange = dailyClose.pct_change()
dailyPctChange.fillna(0, inplace=True)

dailyReturns_log = np.log(dailyClose.pct_change()+1)


In [12]:
#add rolling averages

wd_short = 5
wd_medium = 20
wd_medium_ema = 13
wd_long = 200

df_close = df_data_dtFilter[["close"]].copy()
df_close["mav_short"] = df_close["close"].rolling(window=wd_short).mean()
df_close["mav_medium"] = df_close["close"].rolling(window=wd_medium).mean()
df_close["mav_long"] = df_close["close"].rolling(window=wd_long).mean()

df_close["ema_short"] = df_close["close"].ewm(span=wd_short,  ignore_na=True).mean()
df_close["ema_med"] = df_close["close"].ewm(span=wd_medium_ema,  ignore_na=True).mean()

In [13]:
# Bollinger Bands
df_close["bohlinger_upper"] = df_close["mav_medium"] + (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))
df_close["bohlinger_lower"] = df_close["mav_medium"] - (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))


In [14]:
# Stochastic Oscillator

so_wd_k = 14
so_wd_d = 3

df_close["k_line"] = ((df_close["close"] - df_close["close"].rolling(window=so_wd_k).min()) * 100 / 
    (df_close["close"].rolling(window=so_wd_k).max() - df_close["close"].rolling(window=so_wd_k).min()))

df_close["d_line"] = df_close["k_line"].rolling(window=so_wd_k).mean()

df_close["so_20"] = 20
df_close["so_80"] = 80

In [15]:
# MACD
macd_fast = 26
macd_slow = 12
macd_smooth = 9

df_close["macd_ema_fast"] = df_close["close"].ewm(span=macd_fast, adjust=False, ignore_na=True).mean()
df_close["macd_ema_slow"] = df_close["close"].ewm(span=macd_slow, adjust=False, ignore_na=True).mean()
df_close["macd_base"] = df_close["macd_ema_fast"] - df_close["macd_ema_slow"]
df_close["macd_sigal"] = df_close["macd_base"].ewm(span=macd_smooth, adjust=False, ignore_na=True).mean()
df_close["macd_hist"] = df_close["macd_base"] - df_close["macd_sigal"]
df_close["macd_color"] = np.where(df_close["macd_hist"]<0, "red", "green")

df_close.tail(5)

,close,mav_short,mav_medium,mav_long,ema_short,ema_med,bohlinger_upper,bohlinger_lower,k_line,d_line,so_20,so_80,macd_ema_fast,macd_ema_slow,macd_base,macd_sigal,macd_hist,macd_color
dateTime,,,,,,,,,,,,,,,,,,
2021-08-13 12:47:00+02:00,"46,340.000","46,357.621","46,403.096","46,260.004","46,359.788","46,386.656","46,422.632","46,383.560",0.000,37.905,20,80,"46,395.317","46,384.656",10.661,-1.887,12.548,green
2021-08-13 12:48:00+02:00,"46,363.898","46,356.463","46,399.262","46,261.051","46,361.158","46,383.405","46,419.402","46,379.122",18.555,38.291,20,80,"46,392.990","46,381.463",11.527,0.796,10.731,green
2021-08-13 12:49:00+02:00,"46,401.859","46,362.832","46,398.731","46,262.240","46,374.725","46,386.041","46,420.482","46,376.979",48.027,38.269,20,80,"46,393.647","46,384.601",9.046,2.446,6.600,green
2021-08-13 12:50:00+02:00,"46,344.898","46,358.609","46,396.611","46,263.114","46,364.783","46,380.164","46,420.404","46,372.819",3.996,32.656,20,80,"46,390.036","46,378.493",11.543,4.266,7.278,green
2021-08-13 12:51:00+02:00,"46,329.129","46,355.957","46,394.908","46,264.177","46,352.898","46,372.873","46,420.744","46,369.072",0.000,27.771,20,80,"46,385.524","46,370.898",14.626,6.338,8.288,green


In [16]:
fig2 = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.01
                    , specs=[[{}],[{}],[{}],[{}],[{"secondary_y": True}]])

### ---------------> OHLC
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
), row=1, col=1)         


### ---------------> ema_short
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_short"]
    , line = dict(color="blue")
    , name="ema_{}".format(wd_short)
), row=1, col=1)

### ---------------> ema_med
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_med"]
    , line = dict(color="yellow")
    , name="ema_{}".format(wd_medium_ema)
), row=1, col=1)


######### 2nd graph
### ---------------> BASE CLOSE
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=5)

    ), row=2, col=1
)

### ---------------> MAV Short
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["mav_short"]
        , name="SMA_{}".format(wd_short)

    ), row=2, col=1
)


### ---------------> MAV medium
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["mav_medium"]
        , line=dict(width=4)
        , name="SMA_{}".format(wd_medium)
    ), row=2, col=1
)

### ---------------> MAV long
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["mav_long"]
        , line=dict(width=2, dash="dot")
        , name="SMA_{}".format(wd_long)
    ), row=2, col=1
)

######### 3rd graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)

    ), row=3, col=1
)

### ---------------> Bohlinger Up
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_upper"]
        , line=dict(dash="dot", color="red")
        , name="bollinger_up"
    ), row=3, col=1
)

### ---------------> Bohlinger Low
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_lower"]
        , line=dict(dash="dot", color="red")
        , name="bollinger_low"
    ), row=3, col=1
)

######### 4th graph
### ---------------> MACD

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_base"]
        , name="MACD"
        , line=dict(color="black", width=2)


    ), row=4, col=1
)

### ---------------> MACD SIGNAL

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_sigal"]
        , name="macd_signal"
        , line=dict(color="lightblue", width=2, dash="dot")

    ), row=4, col=1
)

### ---------------> MACD HIST

fig2.add_trace(
    go.Bar(
        x=df_close.index
        , y=df_close["macd_hist"]
        , name="macd_hist"
        , marker_color=df_close["macd_color"]

    ), row=4, col=1
)

######### 5th graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
        , yaxis="y11"

    ), row=5, col=1
)

### ---------------> %K
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["k_line"]
        , name="%K"
        , line=dict(color="red", width=0.5, dash="solid")
        

    ), row=5, col=1, secondary_y=True
)

### ---------------> SO bounds
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_20"]
        , name="bound_20"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=5, col=1, secondary_y=True
)

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_80"]
        , name="bound_80"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=5, col=1, secondary_y=True
)


rn_yaxis_a = 44000
rn_yaxis_b = 47000

fig2.update_layout(
    width=1500
    , height=2000
    , xaxis1 = dict(rangeslider=dict(visible=False))
    , xaxis5 = dict(rangeslider=dict(visible=True))
    , yaxis1 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis2 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis3 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis5 = dict(range=[rn_yaxis_a,rn_yaxis_b])

)


fig2.update_xaxes(matches="x"
    ,rangebreaks=[dict(values=l_skipDates, dvalue=(60* 1000))]
)

fig2.show()

In [17]:
df_ohlc = df_data_dtFilter[["open","high","low","close"]].copy()

df_ohlc["F_ema_5"] = ta.EMA(df_ohlc, 5)
df_ohlc["F_ema_13"] = ta.EMA(df_ohlc, 13)
df_ohlc[-20:]

,open,high,low,close,F_ema_5,F_ema_13
dateTime,,,,,,
2021-08-13 12:32:00+02:00,"46,365.199","46,406.828","46,354.512","46,406.828","46,403.106","46,413.131"
2021-08-13 12:33:00+02:00,"46,406.828","46,406.840","46,383.000","46,387.281","46,397.831","46,409.438"
2021-08-13 12:34:00+02:00,"46,387.289","46,388.512","46,380.012","46,380.012","46,391.891","46,405.234"
2021-08-13 12:35:00+02:00,"46,380.012","46,428.559","46,380.012","46,425.000","46,402.928","46,408.058"
2021-08-13 12:36:00+02:00,"46,425.180","46,473.262","46,418.551","46,468.801","46,424.885","46,416.735"
2021-08-13 12:37:00+02:00,"46,470.559","46,477.102","46,441.422","46,450.648","46,433.473","46,421.580"
2021-08-13 12:38:00+02:00,"46,450.621","46,462.770","46,440.000","46,461.000","46,442.649","46,427.211"
2021-08-13 12:39:00+02:00,"46,460.988","46,467.590","46,457.969","46,462.578","46,449.292","46,432.264"
2021-08-13 12:40:00+02:00,"46,462.578","46,463.238","46,395.359","46,402.621","46,433.735","46,428.029"
